<a href="https://colab.research.google.com/github/AmirAziz1221/AmirAziz1221/blob/main/Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Training the Water Segmentation Model

In [1]:
# Install and Import Libraries
!pip install ultralytics roboflow

from ultralytics import YOLO
from roboflow import Roboflow


# Train the YOLOv8 segmentation model
model = YOLO("yolov8n-seg.pt")  # YOLOv8 segmentation pre-trained model

# Train the model
model.train(
    data=f"/content/drive/MyDrive/Water Segmentation/Water Segmenting v1.v1-version-1.yolov11/data.yaml",  # Path to the dataset configuration
    epochs=20,                             # Increase epochs for better performance
    imgsz=640,                             # Image size
    device=0                               # Use GPU if available
)

# Save this model

In [ ]:
model.save("26_best_water_segmentation_model.pt")

# Evaluating the Model on a Video

In [1]:
# Install required libraries
import cv2
import numpy as np
import torch
from ultralytics import YOLO  # Make sure to import the YOLO model from the appropriate library

# Load the trained model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLO("26_best_water_segmentation_model.pt")

# Path to the input video
video_path = "/content/drive/MyDrive/Water Segmentation/combine video.mp4"
cap = cv2.VideoCapture(video_path)

# Output video writer setup
output_path = "/content/drive/MyDrive/Water Segmentation/26_segmented_output.mp4"
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Original video FPS
frame_width, frame_height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    input_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model.predict(input_frame, imgsz=640)  # Run inference

    # Check if masks are available
    if results[0].masks is not None:
        # Extract the first segmentation mask
        mask = results[0].masks.data[0].cpu().numpy()  # Get the first mask
        mask = (mask > 0.5).astype(np.uint8) * 255  # Binarize mask

        # Overlay the mask on the frame
        overlay = cv2.addWeighted(frame, 0.8, cv2.merge((mask, mask, mask)), 0.2, 0)
    else:
        # If no mask is detected, use the original frame
        overlay = frame

    # Write the frame to the output video
    out.write(overlay)

# Release resources
cap.release()
out.release()
print("Video processing and saving completed.")

# Short video processing

In [ ]:
# Install required libraries
import cv2
import numpy as np
import torch

# Load the trained model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLO("26_best_water_segmentation_model.pt")  # Load trained model

# Path to the input video
video_path = "/content/drive/MyDrive/Water Segmentation/combine video.mp4"
cap = cv2.VideoCapture(video_path)

# Output video writer setup
output_path = "/content/drive/MyDrive/Water Segmentation/short_segmented_output.mp4"
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Original video FPS
frame_width, frame_height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Define frame skipping parameters
frame_skip_rate = 1000  # Process every 1000th frame
frame_counter = 0       # Initialize frame counter
processed_frames = 0    # Keep track of processed frames
max_frames_to_process = fps * 60 * 5  # Target duration: 5 minutes (based on FPS)

# Process the video
while cap.isOpened() and processed_frames < max_frames_to_process:
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1
    if frame_counter % frame_skip_rate != 0:
        continue

    # Preprocess the frame
    input_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model.predict(input_frame, imgsz=640)  # Run inference

    # Check if masks are detected
    if results[0].masks is not None:
        mask = results[0].masks.data[0].cpu().numpy()  # Get the first mask
        mask = (mask > 0.5).astype(np.uint8) * 255  # Binarize mask

        # Overlay the mask on the frame
        overlay = cv2.addWeighted(frame, 0.8, cv2.merge((mask, mask, mask)), 0.2, 0)

        # Write the frame to the output video
        out.write(overlay)
        processed_frames += 1
    else:
        print(f"No segmentation mask detected for frame {frame_counter}.")

# Release resources
cap.release()
out.release()
print("Short video processing and saving completed.")


0: 384x640 (no detections), 15.1ms
Speed: 3.7ms preprocess, 15.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
No segmentation mask detected for frame 1000.

0: 384x640 (no detections), 12.9ms
Speed: 3.4ms preprocess, 12.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
No segmentation mask detected for frame 2000.

0: 384x640 (no detections), 10.8ms
Speed: 4.0ms preprocess, 10.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
No segmentation mask detected for frame 3000.

0: 384x640 (no detections), 19.0ms
Speed: 4.6ms preprocess, 19.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
No segmentation mask detected for frame 4000.

0: 384x640 (no detections), 12.5ms
Speed: 5.9ms preprocess, 12.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
No segmentation mask detected for frame 5000.

0: 384x640 (no detections), 17.4ms
Speed: 3.5ms preprocess, 17.4ms inference, 0.7ms postprocess per imag